# GEMdataset

> This package will hold the GxExM dataset.

In [ ]:
#| default_exp GEMlearn

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import torch
from torch.utils.data import DataLoader
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer

from sklearn.preprocessing import StandardScaler
from pathlib import Path

import fastcore.all as fc
from collections.abc import Mapping
from pathlib import Path
from operator import attrgetter,itemgetter
from functools import partial
from copy import copy
from contextlib import contextmanager
from warnings import warn
import torch.nn.functional as F
import torch.optim as optim
import torch.nn as nn
from torcheval.metrics import MeanSquaredError,Mean, R2Score
from fastprogress import progress_bar,master_bar

from hybridpredictmaize22.GEMdataset import *

In [ ]:
#| export
def remove_leapdays(weather):
    """ just a hotfix """
    to_remove = []
    for i in list(set(weather_data['Env'])):
        if (sum(weather_data['Env'] == i)) == 366:
            #get indexes
            to_remove.append(max(list(weather_data.loc[weather_data['Env'] == i].index)))
    return weather_data.drop(to_remove)

In [ ]:
#| export
class DataLoaders:
    def __init__(self, *dls): self.train,self.valid = dls[:2]

    @classmethod
    def from_dd(cls, dd, batch_size, as_tuple=True, **kwargs):
        return cls(*[DataLoader(ds, batch_size, collate_fn=collate_dict(ds), **kwargs) for ds in dd.values()])

In [ ]:
#| export
class CancelFitException(Exception): pass
class CancelBatchException(Exception): pass
class CancelEpochException(Exception): pass

In [ ]:
#| export
def run_cbs(cbs, method_nm, learn=None):
    for cb in sorted(cbs, key=attrgetter('order')):
        method = getattr(cb, method_nm, None)
        if method is not None: method(learn)

In [ ]:
#| export
class Callback(): order = 0

In [ ]:
#| export
class CompletionCB(Callback):
    def before_fit(self, learn): self.count = 0
    def after_batch(self, learn): self.count += 1
    def after_fit(self, learn): print(f'Completed {self.count} batches')

In [ ]:
#| export
def to_cpu(x):
    if isinstance(x, Mapping): return {k:to_cpu(v) for k,v in x.items()}
    if isinstance(x, list): return [to_cpu(o) for o in x]
    if isinstance(x, tuple): return tuple(to_cpu(list(x)))
    return x.detach().cpu()

In [ ]:
#| export
@contextmanager
def callback_ctx(cbmeth, nm):
    try:
        cbmeth(f'before_{nm}')
        yield
        cbmeth(f'after_{nm}')
    except globals()[f'Cancel{nm.title()}Exception']: pass
    finally: cbmeth(f'cleanup_{nm}')

In [ ]:
#| export
class MetricsCB(Callback):
    def __init__(self, *ms, **metrics):
        for o in ms: metrics[type(o).__name__] = o
        self.metrics = metrics
        self.all_metrics = copy(metrics)
        self.all_metrics['loss'] = self.loss = Mean()

    def _log(self, d): print(d)
    def before_fit(self, learn): learn.metrics = self
    def before_epoch(self, learn): [o.reset() for o in self.all_metrics.values()]

    def after_epoch(self, learn):
        log = {k:f'{v.compute():.3f}' for k,v in self.all_metrics.items()}
        log['epoch'] = learn.epoch
        log['train'] = 'train' if learn.model.training else 'eval'
        self._log(log)

    def after_batch(self, learn):
        y,_,_ = to_cpu(learn.batch)
        if y.shape == learn.preds.squeeze().shape:
            for m in self.metrics.values(): m.update(to_cpu(learn.preds.squeeze()), y)
            self.loss.update(to_cpu(learn.loss), weight=len(y))
        else:
            pass

In [ ]:
#| export
class ProgressCB(Callback):
    order = MetricsCB.order+1
    def __init__(self, plot=False): self.plot = plot
    def before_fit(self, learn):
        learn.epochs = self.mbar = master_bar(learn.epochs)
        self.first = True
        if hasattr(learn, 'metrics'): learn.metrics._log = self._log
        self.losses = []

    def _log(self, d):
        if self.first:
            self.mbar.write(list(d), table=True)
            self.first = False
        self.mbar.write(list(d.values()), table=True)

    def before_epoch(self, learn): learn.dl = progress_bar(learn.dl, leave=False, parent=self.mbar)
    def after_batch(self, learn):
        learn.dl.comment = f'{learn.loss:.3f}'
        if self.plot and hasattr(learn, 'metrics') and learn.training:
            self.losses.append(learn.loss.item())
            self.mbar.update_graph([[fc.L.range(self.losses), self.losses]])

In [ ]:
#| export
class Learner():
    def __init__(self, model, dls=(0,), loss_func=F.mse_loss, lr=0.1, cbs=None, opt_func=optim.SGD):
        cbs = fc.L(cbs)
        self.cb_ctx = partial(callback_ctx, self.callback)
        fc.store_attr()

    def one_epoch(self, train):
        self.model.train(train)
        self.dl = self.dls.train if train else self.dls.valid
        with self.cb_ctx('epoch'):
            for self.iter,self.batch in enumerate(self.dl):
                with self.cb_ctx('batch'):
                    self.predict()
                    self.get_loss()
                    if self.training:
                        self.backward()
                        self.step()
                        self.zero_grad()
    
    def fit(self, n_epochs=1, train=True, valid=True, cbs=None, lr=None):
        cbs = fc.L(cbs)
        # `add_cb` and `rm_cb` were added in lesson 18
        for cb in cbs: self.cbs.append(cb)
        try:
            self.n_epochs = n_epochs
            self.epochs = range(n_epochs)
            self.opt = self.opt_func(self.model.parameters(), self.lr if lr is None else lr)
            with self.cb_ctx('fit'):
                for self.epoch in self.epochs:
                    if train: self.one_epoch(True)
                    if valid: torch.no_grad()(self.one_epoch)(False)
        finally:
            for cb in cbs: self.cbs.remove(cb)

    def __getattr__(self, name):
        if name in ('predict','get_loss','backward','step','zero_grad'): return partial(self.callback, name)
        raise AttributeError(name)

    def callback(self, method_nm): run_cbs(self.cbs, method_nm, self)
    
    @property
    def training(self): return self.model.training

In [ ]:
#| export
class OneDCNN(torch.nn.Module):
    def __init__(self):
        super(OneDCNN, self).__init__()

        # Define the layers
        self.conv1 = nn.Conv1d(in_channels=16, out_channels=100, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(in_channels=100, out_channels=200, kernel_size=3, padding=1)
        self.conv3 = nn.Conv1d(in_channels=200, out_channels=400, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(146000, 100)
        self.fc2 = nn.Linear(100, 100)
        self.dropout = nn.Dropout(p=0.5)
        self.relu = nn.ReLU()

    def forward(self, x):
        # Apply the layers
        x =  x.reshape(x.shape[0],1,50)
        x = self.conv1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.conv3(x)
        x = self.relu(x)
        x = self.relu(x)
        x = x.view(x.shape[0],x.shape[1]*x.shape[2])
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)

        return x
    

class MLP(torch.nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = torch.relu(x)
        x = self.fc2(x)
        return x


class GxE(torch.nn.Module):
    def __init__(self,mlp, cnn):
        super(GxE, self).__init__()
        self.MLP = mlp
        self.OneDCNN = cnn
        self.fc1 = nn.Linear(200,100)
        self.fc2 = nn.Linear(100,1)
        
    def forward(self,x):
        g,w = x
#        g = g.reshape(batch_size,100)
        
        
        g = self.MLP(g)
        w = self.OneDCNN(w)
        
        x= torch.concat((g,w),axis=1)
        x = self.fc1(x)
        x = torch.relu(x)
        #print(x.shape)
        x = self.fc2(x)
        return x

In [ ]:
#| export

def_device = 'cuda' if torch.cuda.is_available() else 'cpu'
def to_device(x, device=def_device):
    if isinstance(x, Mapping): return {k:v.to(device) for k,v in x.items()}
    return type(x)(o.type(torch.float32).to(device) for o in x)

In [ ]:
#| export
class TrainCB(Callback):
    def predict(self, learn):
        learn.preds = learn.model(learn.batch[1])
    def get_loss(self, learn):
        #print(learn.preds.squeeze().shape, learn.batch[0].shape)
        learn.loss = learn.loss_func(learn.preds.squeeze(), learn.batch[0])
    def backward(self, learn): learn.loss.backward()
    def step(self, learn): learn.opt.step()
    def zero_grad(self, learn): learn.opt.zero_grad()

In [ ]:
#| export
def_device = 'cuda' if torch.cuda.is_available() else 'cpu'
class DeviceCB(Callback):
    def __init__(self, device=def_device): fc.store_attr()
    def before_fit(self, learn):
        learn.model.to(self.device)
    def before_batch(self, learn): learn.batch = to_device(learn.batch, device=self.device)

In [ ]:
#| hide
# import random
# #generate random SNP matrix
# gene_dosages = [0, .5, 1]
# years = [2018,2019]
# snp_length = 100
# number_hybrids = 20


# number_environments = 10
# env_col = []
# for i,y in zip(np.arange(number_environments),[random.choice(years) for _ in range(number_environments)]):
#     env_col.append(f'{i}_{y}')

# snp_matrix = (np.arange(number_hybrids),np.array([[random.choice(gene_dosages) for x in range(snp_length)] for _ in range(number_hybrids)]))

# #generate random yield data
# random_yields = [random.uniform(-1,1) for _ in range(100)]
# random_hybrids = [random.choice(range(number_hybrids)) for _ in range(100)]
# random_environments = [random.choice((env_col)) for _ in range(100)]

# yield_data = pd.DataFrame({"Hybrid":random_hybrids, "Yield_Mg_ha":random_yields, 'Env':random_environments})
# yield_data.head()

# Weather_Table = np.random.random((50,number_environments))
# weather_table = {}
# for c,i in enumerate(Weather_Table):
#     weather_table[c] = i
    
# weather_data = pd.DataFrame(weather_table)
# weather_data.insert(0,'Env',env_col)
# weather_data.insert(1,'Year',[x.split('_')[1] for x in env_col])

# weather_data

# #Create a GEM dataset
# test_split = 2019
# gem = GEM(test_split)
# gem.Y = YT(yield_data, test_split)
# gem.W = WT(weather_data, test_split)
# gem.SNP = snp_matrix

# ds = GemDataset(gem.W.Tr, gem.Y.Tr, gem.SNP)
# tr_ds = GemDataset(gem.W.Tr, gem.Y.Tr, gem.SNP)
# te_ds = GemDataset(gem.W.Te, gem.Y.Te, gem.SNP)
# tr_dl = DataLoader(tr_ds, batch_size=4)
# te_dl = DataLoader(te_ds, batch_size=4)
# dls = DataLoaders(tr_dl,te_dl)

# metrics = MetricsCB(MeanSquaredError())
# model = MLP(20,100, 1)
# cbs = [TrainCB(), DeviceCB(), metrics, ProgressCB(plot=True)]
# learn = Learner(model, dls, F.mse_loss, lr=.25, cbs=cbs)
# learn.fit(1)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()